# Advanced Querying Mongo

Importing libraries and setting up connection

In [19]:
from pymongo import MongoClient
from pymongo import ASCENDING, DESCENDING
dbName = "companies"
mongodbURL = f"mongodb://localhost/{dbName}"
print(mongodbURL)
client = MongoClient(mongodbURL)
db = client.get_database()

mongodb://localhost/companies


### 1. All the companies whose name match 'Babelgum'. Retrieve only their `name` field.

In [13]:
cursor = db.companies.find({'name':'Babelgum'}, {"name":1})
for e in cursor:
    print(e['name'])

Babelgum


### 2. All the companies that have more than 5000 employees. Limit the search to 20 companies and sort them by **number of employees**.

In [26]:
cursor = db.companies.find({'number_of_employees':{'$gt':5000}}).sort([('number_of_employees', DESCENDING)]).limit(20)
for e in cursor:
    print(e['name'], e['number_of_employees'])

Siemens 405000
IBM 388000
Toyota 320000
PayPal 300000
Nippon Telegraph and Telephone Corporation 227000
Samsung Electronics 221726
Accenture 205000
Tata Consultancy Services 200300
Flextronics International 200000
Safeway 186000
Sony 180500
LG 177000
Ford 171000
Boeing 160000
Digital Equipment Corporation 140000
Nokia 125000
MItsubishi Electric 107000
MItsubishi Electric 107000
Comcast 100000
Bertelsmann 100000


### 3. All the companies founded between 2000 and 2005, both years included. Retrieve only the `name` and `founded_year` fields.

In [30]:
#Limited to 10 for brevity
years = list(range(2000, 2006))
cursor = db.companies.find({'founded_year':{'$in':years}}, {'name': 1, 'founded_year': 1}).limit(10)
for e in cursor:
    print(e['name'], e['founded_year'])

Zoho 2005
Wetpaint 2005
Omnidrive 2005
Digg 2004
Gizmoz 2003
StumbleUpon 2002
Facebook 2004
Helio 2005
Plaxo 2002
Technorati 2002


### 4. All the companies that had a Valuation Amount of more than 100.000.000 and have been founded before 2010. Retrieve only the `name` and `ipo` fields.

In [41]:
cursor = db.companies.find({'$and':[{'ipo.valuation_amount':{'$gt': 100000000}},
                                    {'founded_year':{'$lt':2010}}]}, 
                           {'name':1,'ipo':1})
for i,e in enumerate(cursor):
    if i == 3: break
    print(e['name'], e['ipo'])

Facebook {'valuation_amount': 104000000000, 'valuation_currency_code': 'USD', 'pub_year': 2012, 'pub_month': 5, 'pub_day': 18, 'stock_symbol': 'NASDAQ:FB'}
Twitter {'valuation_amount': 18100000000, 'valuation_currency_code': 'USD', 'pub_year': 2013, 'pub_month': 11, 'pub_day': 7, 'stock_symbol': 'NYSE:TWTR'}
Yelp {'valuation_amount': 1300000000, 'valuation_currency_code': 'USD', 'pub_year': 2012, 'pub_month': 3, 'pub_day': 2, 'stock_symbol': 'NYSE:YELP'}


### 5. All the companies that have less than 1000 employees and have been founded before 2005. Order them by the number of employees and limit the search to 10 companies.

In [ ]:
cursor = db.companies.find({}

### 6. All the companies that don't include the `partners` field.

In [ ]:
# Your Code

### 7. All the companies that have a null type of value on the `category_code` field.

In [ ]:
# Your Code

### 8. All the companies that have at least 100 employees but less than 1000. Retrieve only the `name` and `number of employees` fields.

In [ ]:
# Your Code

### 9. Order all the companies by their IPO price in a descending order.

In [ ]:
# Your Code

### 10. Retrieve the 10 companies with more employees, order by the `number of employees`

In [ ]:
# Your Code

### 11. All the companies founded on the second semester of the year. Limit your search to 1000 companies.

In [ ]:
# Your Code

### 12. All the companies founded before 2000 that have an acquisition amount of more than 10.000.00

In [ ]:
# Your Code

### 13. All the companies that have been acquired after 2010, order by the acquisition amount, and retrieve only their `name` and `acquisition` field.

In [ ]:
# Your Code

### 14. Order the companies by their `founded year`, retrieving only their `name` and `founded year`.

In [ ]:
# Your Code

### 15. All the companies that have been founded on the first seven days of the month, including the seventh. Sort them by their `acquisition price` in a descending order. Limit the search to 10 documents.

In [ ]:
# Your Code

### 16. All the companies on the 'web' `category` that have more than 4000 employees. Sort them by the amount of employees in ascending order.

In [ ]:
# Your Code

### 17. All the companies whose acquisition amount is more than 10.000.000, and currency is 'EUR'.

In [ ]:
# Your Code

### 18. All the companies that have been acquired on the first trimester of the year. Limit the search to 10 companies, and retrieve only their `name` and `acquisition` fields.

In [ ]:
# Your Code

# Bonus
### 19. All the companies that have been founded between 2000 and 2010, but have not been acquired before 2011.

In [ ]:
# Your Code

### 20. All the companies that have been 'deadpooled' after the third year.

In [ ]:
# Your Code